In [1]:
import warnings
warnings.filterwarnings("ignore")
import PIL

## TRANSFORMACION DE .PCAP -> .BIN

In [ ]:
import os
from scapy.all import rdpcap, IP, UDP, TCP

# Parámetros de configuración
HASH_SIZE = 20000  # Tamaño de la tabla hash para los flujos
MAX_PAYLOAD_SIZE = 1024  # Máximo tamaño del paquete a guardar

# Tabla hash simulada para flujos
hash_table = [None] * HASH_SIZE
contador_flujos = 0

# Función para crear un directorio
def create_output_directory(output_dir, flow_id):
    """
    Crea un directorio para un flujo específico dentro de la carpeta de salida especificada.
    """
    flow_dir = os.path.join(output_dir, str(flow_id))
    os.makedirs(flow_dir, exist_ok=True)
    return flow_dir

# Función para guardar paquetes en archivos
def save_packet_to_file(flow_dir, packet_data, packet_id):
    """
    Guarda los datos de un paquete en un archivo dentro del flujo correspondiente.
    """
    file_path = os.path.join(flow_dir, f"{packet_id}.bin")
    with open(file_path, 'wb') as file:
        file.write(packet_data[:MAX_PAYLOAD_SIZE])

# Función para analizar un paquete
def analizar_paquete(paquete, output_dir, token, contador):
    """
    Analiza un paquete y guarda el contenido directamente en la carpeta del archivo PCAP.
    """
    if IP not in paquete: #no se sobre esto tbh lol xdxdxd
        print("No es un paquete IP. Descartado.")
        return

    payload = bytes(paquete[IP].payload)
    file_path = os.path.join(output_dir, f"{token}_{contador}.bin")
    with open(file_path, 'wb') as file:
        file.write(payload[:MAX_PAYLOAD_SIZE])

# Función principal para procesar múltiples PCAPs
def procesar_capturas(input_dir, output_dir):
    """
    Procesa todos los archivos PCAP en un directorio de entrada y genera resultados en subcarpetas del directorio de salida.
    """
    global contador_flujos
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Procesar cada archivo PCAP en la carpeta de entrada
    for filename in os.listdir(input_dir):
        if not filename.endswith(".pcap"):
            continue

        pcap_path = os.path.join(input_dir, filename)
        if not os.path.isfile(pcap_path):
            continue

        # Crear una subcarpeta en el directorio de salida con el nombre del archivo (sin extensión)
        token = os.path.splitext(filename)[0]
        file_output_dir = os.path.join(output_dir, token)
        os.makedirs(file_output_dir, exist_ok=True)

        print(f"Procesando archivo: {filename}...")

        # Leer y analizar paquetes
        paquetes = rdpcap(pcap_path)
        contador = 0
        for paquete in paquetes:
            contador += 1
            analizar_paquete(paquete, file_output_dir, token, contador)

        print(f"Archivo procesado. Resultados en: {file_output_dir}")

In [47]:
## PASAR DE .PCAP A .BIN
INPUT_DIR = "../data/inputs/filtered/"
OUTPUT_DIR = "../data/outputs/binary/"

procesar_capturas(INPUT_DIR, OUTPUT_DIR)

Procesando archivo: f_vpn_aim_chat1a.pcap...
Archivo procesado. Resultados en: ../data/outputs/binary/f_vpn_aim_chat1a
Procesando archivo: f_vpn_aim_chat1b.pcap...
Archivo procesado. Resultados en: ../data/outputs/binary/f_vpn_aim_chat1b
Procesando archivo: f_vpn_bittorrent.pcap...
Archivo procesado. Resultados en: ../data/outputs/binary/f_vpn_bittorrent
Procesando archivo: f_vpn_email2a.pcap...
Archivo procesado. Resultados en: ../data/outputs/binary/f_vpn_email2a
Procesando archivo: f_vpn_email2b.pcap...
Archivo procesado. Resultados en: ../data/outputs/binary/f_vpn_email2b
Procesando archivo: f_vpn_facebook_audio2.pcap...
Archivo procesado. Resultados en: ../data/outputs/binary/f_vpn_facebook_audio2
Procesando archivo: f_vpn_facebook_chat1a.pcap...
Archivo procesado. Resultados en: ../data/outputs/binary/f_vpn_facebook_chat1a
Procesando archivo: f_vpn_facebook_chat1b.pcap...
Archivo procesado. Resultados en: ../data/outputs/binary/f_vpn_facebook_chat1b
Procesando archivo: f_vpn_ftps

## IMAGENES CON CURVA DE MORTON

In [50]:
# ==========================
# Función para aplicar Curva de Morton
# ==========================
def apply_morton_curve(data, width, height):
    """Reorganiza los datos 1D según la Curva de Morton (Z-order) para un arreglo 2D."""
    def interleave_bits(x, y):
        """Interleave the bits of x and y to calculate Morton code."""
        z = 0
        for i in range(max(x.bit_length(), y.bit_length())):
            z |= ((x >> i) & 1) << (2 * i + 1)  # x bits
            z |= ((y >> i) & 1) << (2 * i)      # y bits
        return z

    # Ajustar los datos al tamaño esperado (rellenar o recortar)
    num_pixels = width * height
    if len(data) < num_pixels:
        data = data + [0] * (num_pixels - len(data))  # Rellenar con ceros
    elif len(data) > num_pixels:
        data = data[:num_pixels]  # Recortar los datos

    # Generar el orden Morton
    morton_order = []
    for y in range(height):
        for x in range(width):
            morton_order.append(interleave_bits(x, y))

    # Reorganizar los datos según el orden de Morton
    morton_data = [0] * num_pixels
    for i, idx in enumerate(morton_order):
        morton_data[i] = data[idx]
    return morton_data



In [ ]:
import os
from PIL import Image

# Función para leer datos binarios
def get_binary_data(filename):
    with open(filename, 'rb') as file:
        return list(file.read())

# Función para definir tamaño de imagen
def get_size_2d(data_length):
    return (32, 32)  # Dimensiones fijas según tus requisitos

# Función para guardar imagen
def save_image(filename, data, size, output_folder, curve_name="default"):
    try:
        image = Image.new('L', size)
        image.putdata(data)
        
        # Asegurar que el directorio de salida exista
        os.makedirs(output_folder, exist_ok=True)
        
        # Guardar la imagen
        output_path = os.path.join(output_folder, os.path.basename(filename) + f"_{curve_name}.png")
        image.save(output_path)
    except Exception as e:
        print(f"❌ Error al guardar imagen: {e}")

# Procesar archivo binario a imagen
def create_greyscale_image(filename, output_folder, curve="default"):
    data = get_binary_data(filename)
    size = get_size_2d(len(data))
    
    if curve == "morton":
        data = apply_morton_curve(data, size[0], size[1])
    
    save_image(filename, data, size, output_folder, curve_name=curve)

# Procesar todos los archivos `.bin` en un directorio
def process_files_in_folder(input_dir, output_dir, curve="default"):
    if not os.path.exists(input_dir):
        print(f"⚠️ Carpeta {input_dir} no existe.")
        return

    # Obtener lista de archivos `.bin`
    bin_files = [os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith(".bin")]
    
    if not bin_files:
        print(f"⚠️ No se encontraron archivos .bin en {input_dir}.")
        return
    
    # Carpeta de salida para este subdirectorio
    output_folder = os.path.join(output_dir, os.path.basename(input_dir))
    
    for bin_file in bin_files:
        create_greyscale_image(bin_file, output_folder, curve=curve)

# Procesar todas las subcarpetas
def process_all_subfolders(parent_input_dir, parent_output_dir, curve="default"):
    if not os.path.exists(parent_input_dir):
        print(f"⚠️ Carpeta principal {parent_input_dir} no existe.")
        return
    
    # Obtener todas las subcarpetas dentro del directorio principal
    subfolders = [os.path.join(parent_input_dir, d) for d in os.listdir(parent_input_dir) if os.path.isdir(os.path.join(parent_input_dir, d))]

    if not subfolders:
        print(f"⚠️ No se encontraron subcarpetas en {parent_input_dir}.")
        return

    for subfolder in subfolders:
        process_files_in_folder(subfolder, parent_output_dir, curve=curve)

# ==========================
# Definir directorios principales y correr el procesamiento
# ==========================
parent_input_directory = r"../data/outputs/binary/"
parent_output_directory = r"../data/outputs/image_morton"

process_all_subfolders(parent_input_directory, parent_output_directory, curve="morton")



## IMAGENES SIN TRANSFORMAR

In [ ]:
### FULL TEST MORTON
import os
from PIL import Image

# Función para leer datos binarios
def get_binary_data(filename):
    with open(filename, 'rb') as file:
        return list(file.read())

# Función para definir tamaño de imagen
def get_size_2d(data_length):
    return (32, 32)  # Dimensiones fijas según tus requisitos

# Función para guardar imagen
def save_image(filename, data, size, output_folder, curve_name="default"):
    try:
        image = Image.new('L', size)
        image.putdata(data)
        
        # Asegurar que el directorio de salida exista
        os.makedirs(output_folder, exist_ok=True)
        
        # Guardar la imagen
        output_path = os.path.join(output_folder, os.path.basename(filename) + f"_{curve_name}.png")
        image.save(output_path)
    except Exception as e:
        print(f"❌ Error al guardar imagen: {e}")

# Procesar archivo binario a imagen
def create_greyscale_image(filename, output_folder, curve="default"):
    data = get_binary_data(filename)
    size = get_size_2d(len(data))
    
    if curve == "morton":
        data = apply_morton_curve(data, size[0], size[1])
    
    save_image(filename, data, size, output_folder, curve_name=curve)

# Procesar todos los archivos `.bin` en un directorio
def process_files_in_folder(input_dir, output_dir, curve="default"):
    if not os.path.exists(input_dir):
        print(f"⚠️ Carpeta {input_dir} no existe.")
        return

    # Obtener lista de archivos `.bin`
    bin_files = [os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith(".bin")]
    
    if not bin_files:
        print(f"⚠️ No se encontraron archivos .bin en {input_dir}.")
        return
    
    # Carpeta de salida para este subdirectorio
    output_folder = os.path.join(output_dir, os.path.basename(input_dir))
    
    for bin_file in bin_files:
        create_greyscale_image(bin_file, output_folder, curve=curve)

# Procesar todas las subcarpetas
def process_all_subfolders(parent_input_dir, parent_output_dir, curve="default"):
    if not os.path.exists(parent_input_dir):
        print(f"⚠️ Carpeta principal {parent_input_dir} no existe.")
        return
    
    # Obtener todas las subcarpetas dentro del directorio principal
    subfolders = [os.path.join(parent_input_dir, d) for d in os.listdir(parent_input_dir) if os.path.isdir(os.path.join(parent_input_dir, d))]

    if not subfolders:
        print(f"⚠️ No se encontraron subcarpetas en {parent_input_dir}.")
        return

    for subfolder in subfolders:
        print(f"🔄 Procesando subcarpeta: {subfolder}")
        process_files_in_folder(subfolder, parent_output_dir, curve=curve)

# ==========================
# Definir directorios principales y correr el procesamiento
# ==========================
parent_input_directory = r"../data/outputs/binary"
parent_output_directory = r"../data/outputs/image_normal"

process_all_subfolders(parent_input_directory, parent_output_directory, curve="default")



🔄 Procesando subcarpeta: ../data/BinariosfinalTrue\f_vpn_aim_chat1a
📂 Procesando: ../data/BinariosfinalTrue\f_vpn_aim_chat1a\f_vpn_aim_chat1a_1.bin
✅ Imagen guardada: ../data/ProcesadosMorton\f_vpn_aim_chat1a\f_vpn_aim_chat1a_1.bin_morton.png
📂 Procesando: ../data/BinariosfinalTrue\f_vpn_aim_chat1a\f_vpn_aim_chat1a_10.bin
✅ Imagen guardada: ../data/ProcesadosMorton\f_vpn_aim_chat1a\f_vpn_aim_chat1a_10.bin_morton.png
📂 Procesando: ../data/BinariosfinalTrue\f_vpn_aim_chat1a\f_vpn_aim_chat1a_100.bin
✅ Imagen guardada: ../data/ProcesadosMorton\f_vpn_aim_chat1a\f_vpn_aim_chat1a_100.bin_morton.png
📂 Procesando: ../data/BinariosfinalTrue\f_vpn_aim_chat1a\f_vpn_aim_chat1a_101.bin
✅ Imagen guardada: ../data/ProcesadosMorton\f_vpn_aim_chat1a\f_vpn_aim_chat1a_101.bin_morton.png
📂 Procesando: ../data/BinariosfinalTrue\f_vpn_aim_chat1a\f_vpn_aim_chat1a_102.bin
✅ Imagen guardada: ../data/ProcesadosMorton\f_vpn_aim_chat1a\f_vpn_aim_chat1a_102.bin_morton.png
📂 Procesando: ../data/BinariosfinalTrue\f_v